----

In [ ]:
# Set-up and importing required libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import matplotlib.ticker as ticker
aspect_ratio = (12,8)
annualization_factor = 260

In [ ]:
# Function to read in the prices data.

def read_price_data(filename, index_col=0, parse_dates=True):
    df = pd.read_csv(filename, index_col=0, parse_dates=True)

    # Printing out the data frame’s info
    print("Data Frame Info:")
    print(df.info())

    print("===================")
    
    # Checking for any null values
    print("Null Values:")
    print(df.isnull().sum())

    print("===================")

    # Prining the summary statistics
    print("Summary Statistics: ")
    print(df.describe())

    print("===================")

    # Displaying the dataframe
    print("Data Frame")
    print(df)

    return df

prices = read_price_data("Prices.csv", index_col=0, parse_dates=True)

----

In [ ]:
# Function for general formatting.

def print_pct_data(data_frame, format_spec="{:.4%}", na_rep="NaN", index_slice=None):
    if index_slice is not None:
        return data_frame.style.format(format_spec, subset=index_slice, na_rep=na_rep)
    else:
        return data_frame.style.format(format_spec, na_rep=na_rep)

# Function for formatting outputs and calculating summary stats. 

def calcSummaryStatistics(return_df, annualization_factor=260, print_screen=True, percent_format_string='{:.2%}'):
    summary_stats = pd.DataFrame([annualization_factor * return_df.mean(),
                                  np.sqrt(annualization_factor) * return_df.std(),
                                  return_df.mean() / return_df.std() * np.sqrt(annualization_factor),
                                  (return_df.cumsum().fillna(0) - np.maximum.accumulate(return_df.cumsum().fillna(0))).min(),
                                  (return_df.dropna() >= 0).sum() / return_df.dropna().shape[0]],
                                index=['Avg. Annual Return', 'Annual Risk', 'Sharpe Ratio', 'Max DD', 'Success Ratio'])

    summary_stats_fmt = print_pct_data(summary_stats, format_spec=percent_format_string,
                                       index_slice=pd.IndexSlice[["Avg. Annual Return", "Annual Risk", "Max DD", "Success Ratio"], :])

    return summary_stats_fmt

----

In [ ]:
# Function to implement moving average crossover strategies - Flat and Short. 

def movingAverageCrossover(df, fast_window, slow_window):
    df_copy = df.copy()

    # Calculate benchmark returns
    df_copy['Benchmark'] = df_copy.iloc[:, 0].pct_change()

    # Apply MA strategies
    df_copy['FastSMA'] = df_copy.iloc[:, 0].rolling(window=fast_window).mean()
    df_copy['SlowSMA'] = df_copy.iloc[:, 0].rolling(window=slow_window).mean()
    
    df_copy.dropna(inplace=True)
    
    df_copy['MAFlat'] = np.where(df_copy['FastSMA'] > df_copy['SlowSMA'], 1, 0)
    df_copy['MAShort'] = np.where(df_copy['FastSMA'] > df_copy['SlowSMA'], 1, -1)

    # Calculate strategy returns
    df_copy[f"{df_copy.columns[0]}-BMK"] = df_copy.iloc[:, 0].pct_change()
    df_copy[f"{df_copy.columns[0]}-MAFlat"] = df_copy['Benchmark'] * df_copy['MAFlat'].shift(1)
    df_copy[f"{df_copy.columns[0]}-MAShort"] = df_copy['Benchmark'] * df_copy['MAShort'].shift(1)

    return df_copy[[f"{df_copy.columns[0]}-BMK", f"{df_copy.columns[0]}-MAFlat", f"{df_copy.columns[0]}-MAShort"]]

----

In [ ]:
# Function to implement Bollinger Bands strategy. 

def bollingerBands(df, lookback, stdev_bands):
    df = df.copy()

    if len(df.columns) > 1:
        raise ValueError('Dataframe should have only 1 column')

    df['Mean'] = df[df.columns[0]].rolling(window=lookback).mean()
    df['Stdev'] = df[df.columns[0]].rolling(window=lookback).std()
    
    df.dropna(inplace=True)
    
    df['Upper'] = df['Mean'] + stdev_bands*df['Stdev']
    df['Lower'] = df['Mean'] - stdev_bands*df['Stdev']

    df.loc[0, 'Position'] = 0
    for i in range(1,len(df)):
        today = df.index[i]
        yesterday = df.index[i-1]

        if df.loc[today, df.columns[0]] >= df.loc[today,'Upper']:
            df.loc[today, 'Position'] = -1
        elif df.loc[today, df.columns[0]] <= df.loc[today,'Lower']:
            df.loc[today, 'Position'] = 1
        elif df.loc[yesterday,'Position'] == -1 and df.loc[today, df.columns[0]] <= df.loc[today,'Mean']:
            df.loc[today, 'Position'] = 0
        elif df.loc[yesterday,'Position'] == 1 and df.loc[today, df.columns[0]] >= df.loc[today,'Mean']:
            df.loc[today, 'Position'] = 0
        else:
            df.loc[today, 'Position'] = df.loc[yesterday,'Position']

    df[f"{df.columns[0]}-BMK"] = df[df.columns[0]].pct_change()
    df[f"{df.columns[0]}-BB"] = df[df.columns[0]].pct_change() * df['Position'].shift(1)

    return df[[f"{df.columns[0]}-BMK", f"{df.columns[0]}-BB"]]

----

### Instruments Selected

#### Equities:
- AAPL, EXC, INTC, PFE, SPY

#### Fixed Income:
- FBNDX

#### Commodity:
- SPGSCI

#### Currency:
- CAD

----

### Rationale for instrument selection

##### 1. Equities:
- AAPL and INTC ar high-yielding tech stocks. Provide opportunities for growth and can benefit from high tech spending, however these can introduce some volatility. 
- SPY being a broad-market ETF can help balance out some of the volatility by providing some stability and diversification across sectors.
- EXC 

##### 2. Fixed Income Instruments:
- FBNDX can help balance out the equities and provide further stability. Lower expense ratio and higher quality funds compared to VBTIX as well.

##### 3. Commodity:
- SPGSCI can act as an inflation hedge owing to its broad commodity make-up.

##### 4. Currency:
- CAD can appreciate with a rise in commodity prices and also adds some balance to the portfolio. 

#### Correlation

FBNDX bonds, SPGSCI commodities, and CAD all exhibit low correlations with equities, providing diversification. If equities decline, bonds continue to hold value, commodities could rise with threats of inflation, and CAD could go up with commodity strength.

----

#### Sensitivity Analysis

In [ ]:
# Checking some instruments with crossover strategies to estimate optimal parameters. 
# Note: We have run the sensitivity analysis for a subset of our instruments to gauge the ideal parameters,
# however we are only displaying it for one instance to avoid clutter in the notebook.

fast_windows = range(5, 110)
slow_windows = range(50, 500, 5)
annualization_factor = 260

# Initializing empty arrays to store sharpes and windows to use later for plotting.

flat_sharpes = []
short_sharpes = []
fast_window = []
slow_window = []


for fw in fast_windows:
    for sw in slow_windows:
        if fw >= sw: 
            continue
            
        df = movingAverageCrossover(prices[['AAPL']], fw, sw)
        df.columns = [f'AAPL-BMK', f'AAPL-MAFlat', f'AAPL-MAShort']
        mean = df[f'AAPL-MAFlat'].mean() * annualization_factor
        std = df[f'AAPL-MAFlat'].std() * np.sqrt(annualization_factor)
        mean1 = df[f'AAPL-MAShort'].mean() * annualization_factor
        std1 = df[f'AAPL-MAShort'].std() * np.sqrt(annualization_factor)
        sharpe = mean / std
        sharpe1 = mean1 / std1

        flat_sharpes.append(sharpe)
        short_sharpes.append(sharpe1)
        fast_window.append(fw)
        slow_window.append(sw)

In [ ]:
plot_1 = plt.scatter(fast_window, slow_window, c=flat_sharpes, cmap='coolwarm_r')

max_idx = flat_sharpes.index(max(flat_sharpes))
plt.scatter(fast_window[max_idx], slow_window[max_idx], color='red', marker='*')
plt.colorbar(plot_1, label='Sharpe Ratio')
plt.title('AAPL Moving Average Crossover - Flat')
plt.xlabel('Fast Window')
plt.ylabel('Slow Window')


plt.legend([f'Fast Win: {fast_window[max_idx]}, Slow Win: {slow_window[max_idx]}, Sharpe Ratios: {round(max(flat_sharpes), 2)}'])
plt.show()

----

In [ ]:
# Parameter estimation for Bollinger Bands strategy.
# Note: We have run the sensitivity analysis for a subset of our instruments to gauge the ideal parameters,
# however we are only displaying it for one instance to avoid clutter in the notebook.

lookbacks = range(2, 50)
bands = range(1, 4)
lookback = []
band = []
bb_sharpes = []

for lb in lookbacks:
    for bd in bands:
        df1 = bollingerBands(prices[['EXC']], lb, bd)
        if not df1[f'EXC-BB'].std(): 
            continue
        mean2 = df1[f'EXC-BB'].mean() * annualization_factor
        std2 = df1[f'EXC-BB'].std() * np.sqrt(annualization_factor)
        sharpe2 = mean2 / std2
        bb_sharpes.append(sharpe2)
        lookback.append(lb)
        band.append(bd)

In [ ]:
plot_3 = plt.scatter(lookback, band, c=bb_sharpes, cmap='coolwarm_r')

max_idx = bb_sharpes.index(max(bb_sharpes))
plt.scatter(lookback[max_idx], band[max_idx], color='red', marker='*')

plt.title('EXC - Bollinger Bands')
plt.xlabel('Lookback Window')
plt.ylabel('Standard Deviation Bands')  
plt.colorbar(plot_3, label = 'Sharpe Ratio')

plt.legend([f'Lookback Window: {lookback[max_idx]}, Bands: {band[max_idx]}, Sharpe Ratio: {round(max(bb_sharpes), 2)}'], loc = (0, 0.25))
plt.show()

----

In [ ]:
# Applying strategies for each instrument, by looping through every instrument for every strategy. 

equity_instruments = ['AAPL', 'EXC', 'INTC', 'PFE', 'SPY']
fixed_income_instruments = ['FBNDX']
commodity_instruments = ['SPGSCI']
currency_instruments = ['CAD']

all_instruments = equity_instruments + fixed_income_instruments + commodity_instruments + currency_instruments

# Initializing empty lists to store results for each instrument. 

crossover_strategy_dfs = []
bollinger_band_strategy_dfs = []
crossover_columns = []
bb_columns = []

# Parameters

fast_window = 11
slow_window = 230
lookback = 22
stdev_bands = 1

# Loop through each instrument.

for instrument in all_instruments:

    df_instrument = prices[[instrument]]

    # Crossover strategies.
    
    df_crossover_strategies = movingAverageCrossover(df_instrument, fast_window, slow_window)
    crossover_strategy_dfs.append(df_crossover_strategies)

    # Bollinger Band strategy.
    
    df_bollinger_band_strategy = bollingerBands(df_instrument, lookback, stdev_bands)
    bollinger_band_strategy_dfs.append(df_bollinger_band_strategy)

    # Storing column names for each strategy separately to construct dataframes.
    
    crossover_columns.extend(df_crossover_strategies.columns)
    bb_columns.extend(df_bollinger_band_strategy.columns)

# Dataframe for crossover strategy returns

merged_crossover_df = pd.concat(crossover_strategy_dfs, axis=1)
merged_crossover_df.columns = crossover_columns  

# Dataframe for Bollinger Bands strategy returns

merged_bb_df = pd.concat(bollinger_band_strategy_dfs, axis=1)
merged_bb_df.columns = bb_columns  



summary_stats_crossover = calcSummaryStatistics(merged_crossover_df)

summary_stats_bb = calcSummaryStatistics(merged_bb_df)

----

In [ ]:
print_pct_data(merged_crossover_df)

----

In [ ]:
print_pct_data(merged_bb_df)

----

In [ ]:
summary_stats_crossover

----

In [ ]:
summary_stats_bb

----

### Instrument-Strategy selection for the final portfolio

- Looking at the sharpe ratios for the different strategies, and by trying various different combinations for our portfolio, we found the following combination to be the best mix of instruments. 

- Not all our selections were based purely on the individual instrument's highest Sharpe Ratio, for example, we found CAD to have ta slightly higher Sharpe ratio for the Bollinger Band strategy, however, our portfolio overall performs better with the Go-Short strategy returns for CAD.

- Final portoflio selections: 

 **AAPL-MAFlat, EXC-BB, INTC-BB, PFE-BB, SPY-MAShort, FBNDX-MAShort, SPGSCI-MAFlat, CAD-MAShort**.

In [ ]:
final_pf = merged_crossover_df[['AAPL-MAFlat']].join([
    merged_bb_df['EXC-BB'], merged_bb_df['INTC-BB'], merged_bb_df['PFE-BB'],
    merged_crossover_df['SPY-MAShort'], merged_crossover_df['FBNDX-MAShort'], merged_crossover_df['SPGSCI-MAFlat'],
    merged_crossover_df['CAD-MAShort']],
    how="inner"
)

In [ ]:
print_pct_data(final_pf)

In [ ]:
final_pf.columns

In [ ]:
myPortfolioColumns = ['AAPL-MAFlat', 'EXC-BB', 'INTC-BB', 'PFE-BB', 'SPY-MAShort', 'FBNDX-MAShort', 'SPGSCI-MAFlat', 'CAD-MAShort']

----

### Correlation

In [ ]:
corr_pf = final_pf.corr()
corr_pf.style.background_gradient(cmap='coolwarm')

- We can observe that most of our instruments exhibit weak positive or negative correlation with the other instruments in the portfolio.
- There are no large swings in either direction, and the instruments not being correlated introduces independence in the portfolio, i.e., the performance of one asset may not be strongly tied to the performance of another, potentially reducing overall portfolio risk and providing the benefits of diversification.

----

In [ ]:
# Sharpe Ratio for equal weight portfolio.

equal_pf = final_pf.copy()

# Calculating daily portfolio returns for the equal-weight portfolio.

equal_pf['EqWt-Portfolio'] = equal_pf.iloc[:, :8].sum(axis=1) / 8  

# Sharpe ratio calculation. 

annualized_portfolio_return = equal_pf['EqWt-Portfolio'].mean() * annualization_factor  
annualized_portfolio_volatility = equal_pf['EqWt-Portfolio'].std() * np.sqrt(annualization_factor)

sharpe_ratio_portfolio = annualized_portfolio_return / annualized_portfolio_volatility
sharpe_ratio_portfolio

In [ ]:
calcSummaryStatistics(equal_pf[['EqWt-Portfolio']])

- Here, we can see that the Sharpe Ratio of **1.18** for the equal-weighted portfolio outperforms every strategy indivdiually. 

In [ ]:
# Single line Sharpe Ratio calculation for the equal Weight portfolio. (extra credit)
# Equal weights are applied implicitly by taking the sum across axis=1 here.

sharpe_ratio_t = ((final_pf.sum(axis=1).mean() * annualization_factor)/ (final_pf.sum(axis=1).std() * np.sqrt(260)))
print("Equal-weighted Portfolio Sharpe Ratio:", sharpe_ratio_t)

----

### Mean-Variance Optimization

In [ ]:
# MVO via a Monte-Carlo Simulation with 5000 paths. 

rng = np.random.default_rng(seed=64)
num_securities = final_pf.shape[1]
paths = 5000
pf_returns = []
pf_risks = []
sharpe_r = []
weight_vec = []

for i in range(paths):
    wts = rng.random(final_pf.shape[1])
    wts /= np.sum(wts)

    pf_return = (annualization_factor * np.dot(wts, final_pf.mean().T))
    pf_risk = np.sqrt(annualization_factor) * np.sqrt(np.dot(np.dot(wts, final_pf.cov()), wts.T))
    sharpe = pf_return / pf_risk

    pf_returns.append(pf_return)
    pf_risks.append(pf_risk)
    sharpe_r.append(sharpe)
    weight_vec.append(wts)

In [ ]:
pf_returns = np.array(pf_returns)
pf_risks = np.array(pf_risks)

In [ ]:
# Using 'coolwarm_r' to reverse the colormap.

plt.scatter(pf_risks, pf_returns, c = sharpe_r, cmap='coolwarm_r')
ax = plt.subplot()
ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0, decimals=0))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0, decimals=0))
plt.xlabel('Risk (%) ')
plt.ylabel('Return (%) ')
plt.plot(pf_risks[np.argmin(pf_risks)], pf_returns[np.argmin(pf_risks)], 'b*', markersize=7.0, label='Min Vol Portfolio:x={:.2%}; y={:.2%}; {:1.2f}'.format(pf_risks[np.argmin(pf_risks)], pf_returns[np.argmin(pf_risks)], pf_returns[np.argmin(pf_risks)]/pf_risks[np.argmin(pf_risks)]))
plt.plot(pf_risks[np.argmax(sharpe_r)], pf_returns[np.argmax(sharpe_r)], 'r*', markersize=7.0, label='Max Sharpe Portfolio:x={:.2%}; y={:.2%}; {:1.2f}'.format(pf_risks[np.argmax(sharpe_r)], pf_returns[np.argmax(sharpe_r)], np.max(sharpe_r)))
plt.colorbar(label='Sharpe Ratio')
plt.legend()

----

In [ ]:
combined_results = pd.DataFrame({'Volatility' : pf_risks, 'Returns' : pf_returns, 'Sharpe Ratio' : sharpe_r})

In [ ]:
combined_results['Sharpe Ratio'].max()

In [ ]:
# Using the weights generated in the simulation for MVO, creating a weight vector.

weight_vec_df = pd.DataFrame(weight_vec, columns=['AAPL-MAFlat', 'EXC-BB', 'INTC-BB', 'PFE-BB', 'SPY-MAShort', 'FBNDX-MAShort', 'SPGSCI-MAFlat', 'CAD-MAShort'])
weight_vec_vol = weight_vec_df.iloc[combined_results['Volatility'].idxmin()]
weight_vec_sharpe = weight_vec_df.iloc[combined_results['Sharpe Ratio'].idxmax()]

weight_vec_final = pd.DataFrame({'Min Vol' : weight_vec_vol, 'Max Sharpe' : weight_vec_sharpe})
print_pct_data(weight_vec_final)

#### Rationalizing the Weight Vectors

- FBNDX and CAD having highest weights for the Minimum Volatility case. FBNDX, being a bond fund, is expected to be stable and low-risk and going short on it provides opportunities to hedge against interest rate risks and thus gets a higher weight while building a risk minimizing portfolio. Going short on currencies like CAD which can be volatile also hedges our portfolio against the associated risks. 
- AAPL has the highest weight for the Maximum Sharpe case. Being a high yield tech sector stock, the stock is expected to rally during bullish periods of the market and can be a reason behind its high weight when maximizing Sharpe.
- SPY holds steady in both cases, implying a clear stability and strength behind the ETF owing to its implicit diversification.


----

The Maximum Sharpe Ratio achieved after performinng Mean Variance Optimization with the specified parameters is **1.37**, which is relatively higher than the observed Sharpe Ratio of the Equal Weight Portfolio, which was **1.18**.

----

In [ ]:
# Using the Max Sharpe weight values from the weight vector and applying the weights to the final portfolio.

max_sharpe_returns = (final_pf * weight_vec_sharpe).sum(axis=1)

final_pf['MaxSharpe'] = max_sharpe_returns

print_pct_data(final_pf)

In [ ]:
calcSummaryStatistics(final_pf)

- From this table, we can see that the Max Sharpe portfolio fares much better when compared to all the other instruments individually.
- It has a considerabbly higher Sharpe ratio of **1.37**, which is consistent with the Maximum Sharpe we achieved in our MVO.
- It also has a very low value for Max Drawdown at **-10.96%** which indicates that it does not go through large fluctuations.

----

### Beta Calculation

In [ ]:
# Fitting a linear regression model with the SPY Benchmark returns keeping the starting dates consistent, 
# and the Max Sharpe Portfolio, to calculate the Beta.

port_Reg = LinearRegression().fit(merged_crossover_df.fillna(0)[['SPY-BMK']], final_pf['MaxSharpe'])

In [ ]:
pf_beta = port_Reg.coef_

In [ ]:
print(f"The Beta for the Max Sharpe portfolio against the S&P500 is: {pf_beta}")

#### Beta Implications

- A Beta of **0.106** vs SPY suggests that our portfolio returns are mostly independent from the overall stock market, with significantly lower volatility (risk) compared to a broad index fund, in this case, the S&P500. This could be beneficial during times of high market volatility. But in strong periods of the market, this low correlation means the protfolio may trail the higher returns of SPY.

----

### Benchmark Returns

In [ ]:
# Portfolio with all benchmark returns.

equal_bmk = merged_crossover_df[['PFE-BMK']].join([
    merged_crossover_df['AAPL-BMK'], merged_crossover_df['INTC-BMK'], merged_crossover_df['SPY-BMK'],
    merged_crossover_df['EXC-BMK'], merged_crossover_df['FBNDX-BMK'], merged_crossover_df['SPGSCI-BMK'],
    merged_crossover_df['CAD-BMK']],
    how="inner"
)

In [ ]:
equal_bmk['Portfolio'] = equal_bmk.iloc[:, :8].sum(axis=1) / 8 
#print_pct_data(equal_bmk)

In [ ]:
calcSummaryStatistics(equal_bmk['Portfolio'])

- Equal weighted portoflio with all Benchmark returns has a Sharpe Ratio of **0.533**.

In [ ]:
# Portfolio with all Go-Flat strategy returns.

equal_flat = merged_crossover_df[['PFE-MAFlat']].join([
    merged_crossover_df['AAPL-MAFlat'], merged_crossover_df['INTC-MAFlat'], merged_crossover_df['SPY-MAFlat'],
    merged_crossover_df['EXC-MAFlat'], merged_crossover_df['FBNDX-MAFlat'], merged_crossover_df['SPGSCI-MAFlat'],
    merged_crossover_df['CAD-MAFlat']],
    how="inner"
)

In [ ]:
equal_flat['Portfolio'] = equal_flat.iloc[:, :8].sum(axis=1) / 8 
#print_pct_data(equal_flat)

In [ ]:
calcSummaryStatistics(equal_flat['Portfolio'])

- Equal weighted portoflio with Go-Flat strategy applied to all instruments' returns has a Sharpe Ratio of **0.846**.

In [ ]:
# Portfolio with all Go-Short returns.

equal_short = merged_crossover_df[['PFE-MAShort']].join([
    merged_crossover_df['AAPL-MAShort'], merged_crossover_df['INTC-MAShort'], merged_crossover_df['SPY-MAShort'],
    merged_crossover_df['EXC-MAShort'], merged_crossover_df['FBNDX-MAShort'], merged_crossover_df['SPGSCI-MAShort'],
    merged_crossover_df['CAD-MAShort']],
    how="inner"
)

In [ ]:
equal_short['Portfolio'] = equal_short.iloc[:, :8].sum(axis=1) / 8 
#print_pct_data(equal_short)

In [ ]:
calcSummaryStatistics(equal_short['Portfolio'])

- Equal weighted portoflio with Go-Short strategy applied to all instruments' returns has a Sharpe Ratio of **0.461**.

In [ ]:
# Portfolio with all Bollinger Bands strategy returns.

equal_bb = merged_bb_df[['PFE-BB']].join([
    merged_bb_df['AAPL-BB'], merged_bb_df['INTC-BB'], merged_bb_df['SPY-BB'],
    merged_bb_df['EXC-BB'], merged_bb_df['FBNDX-BB'], merged_bb_df['SPGSCI-BB'],
    merged_bb_df['CAD-BB']],
    how="inner"
)

In [ ]:
equal_bb['Portfolio'] = equal_bb.iloc[:, :8].sum(axis=1) / 8  
#print_pct_data(equal_bb)

In [ ]:
calcSummaryStatistics(equal_bb['Portfolio'])

- Equal weighted portoflio with Bollinger Bands strategy applied to all instruments' returns has a Sharpe Ratio of **0.161**.

----

- Note: None of these 4 portfolios outperform the Max Sharpe Portoflio, which had a Sharpe Ratio of **1.37**.

- Among these 4, the portfolio with all instruments run through the Go-Flat strategy has the highest Sharpe Ratio of **0.846**